In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
from pfs.lam.imageAnalysis import neighbor_outlier_filter

from pfs.lam.detAnalysis import *
from pfs.lam.detFocusAnalysis import *
from pfs.lam.fileHandling import *
from pfs.lam.style import *
from pfs.lam.analysisPlot import *
import glob
import lsst.daf.persistence as dafPersist

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
#invMat = "/home/fmadec/devel/ait-notebook/fmadec/Subaru/SM1/SM1_R1_DetFocusInvMat_20190423_F2.8.mat"
invMat="/home/fmadec/devel/lam_sps_devel/notebooks/devel/fmadec/SM2/ws_slit/InvMat_sm1_B1_02oct2020.mat"
#invMat = "SM1_B1_DetFocusInvMat_20190305_bundlesBGR3R6.mat"

#invMat = "/home/pfs/dev/ait-notebook/fmadec/Subaru/SM1/cluster/InvMat_sm1_17sept2020.mat"


In [ ]:
datetime.now().strftime("%Y-%m-%dT%Hh%M")

In [ ]:
roi_size = 24
doBck = True
threshold = 0.

In [ ]:
100*(r1/r)

In [ ]:
experimentId = 255
rerun = "sm1-march2020" #"sm1-march2020"
cam = "m1"
#thres_neighbor = -0.05
extra = 'stack/'

In [ ]:
experimentId = 186
rerun = "sm1-march2020-no_FD" #"sm1-march2020"
cam = "r1"
extra = ''

In [ ]:
experimentId = 122
rerun = "dcb2" #"sm1-march2020"
cam = "b2"
extra = ''

In [ ]:
experimentId = 125
rerun = "defocus" #"sm1-march2020"
cam = "b2"
extra = ''

In [ ]:
expIds = [124,125]
rerun = "defocus" #"sm1-march2020"
cam = "b2"
extra = ''

In [ ]:
expIds = [134]
rerun = "defocus" #"sm1-march2020"
cam = "r2"
extra = ''

In [ ]:
expIds = [152]
rerun = "defocus" #"sm1-march2020"
cam = "r2"
extra = ''

In [ ]:
expIds = [153]
rerun = "defocus" #"sm1-march2020"
cam = "r2"
extra = ''

In [ ]:
# WS_Slit
expIds = [203]
rerun = "ws_slit" #"sm1-march2020"
cam = "b3"
extra = ''
roi_size = 60

# define defaut parameters


In [ ]:
piston_index = "motor1"

com = True  # Center Of Mass
head = 0
tail = 0
criteria = 'EE5'
criterias = [criteria] #['EE5', 'sep_ECE5']
verbose = True
doPrint = False
arm = cam[0]


In [ ]:
head = 0
tail = 0

In [ ]:
cluster = False

In [ ]:
doForestFilter = False
forest_contamin = 0.05   #r1 0.05   #m1 

In [ ]:
doNeighborFilter = True
thres_neighbor = -0.1 # default -0.1

In [ ]:
doSave = False
doSaveCsv = False

# Set folder according place either cluster, laptop

In [ ]:
if cluster :
    imgPath = '/net/SRVSTK20C/drp/fmadec/analysis/subaru/sm1/'
    dataPath = "/net/SRVSTK20C/drp/fmadec/analysis/subaru/sm1/"
    drpPath = "/net/SRVSTK20C/drp/cluster"
    repo = "repo"
else:
    imgPath = "/media/fmadec/PfsData/PFS/fmadec/analysis/subaru/sm1/"
    #imgPath = '/home/fmadec/Pictures/analysis/'
    dataPath = imgPath
    drpPath = "/home/fmadec/pfs/subaruData/"    
    repo = "drp"

if False:
    imgPath = "/home/fmadec/Pictures/analysis/"
    #imgPath = '/home/fmadec/Pictures/analysis/'
    dataPath = imgPath
    drpPath = "/home/fmadec/pfs/subaruData/"    
    repo = "drp"
 #   Exp255/sm1-march2020_roi24/doBckTrue/stack
    
if True:
    imgPath = '/data/drp/analysis/sm2/'
    #imgPath = '/home/fmadec/Pictures/analysis/'
    dataPath = imgPath
    drpPath = "/data/drp"    
    repo = "sps"
 #   Exp255/sm1-march2020_roi24/doBckTrue/stack



In [ ]:
outpath = ''
outpath = "output\\" if outpath is None else outpath



# Read data CSV files for all experimentIds

In [ ]:
files = []
for experimentId in expIds:

    csvPath = imgPath+"Exp"+str(experimentId)+"/"+rerun+"_roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra
    if not os.path.exists(csvPath):
        csvPath = imgPath+"Exp"+str(experimentId)+"/"+rerun+"/"+"roi"+str(roi_size)+"/doBck"+str(doBck)+"/"+extra

    dataPath = csvPath
    print(dataPath)

    searchFile = f"{csvPath}Imquality_{cam}_Exp{experimentId}*"
    print(searchFile)
    files.extend(glob.glob(searchFile))

if verbose:
    print(*files, sep="\n")

piston_imdata = pd.concat(map(pd.read_csv,files)).reset_index().sort_values(by="motor1")
#piston_imdata["wavelength"] = 300

piston_imdata = piston_imdata.groupby(['wavelength','fiber', "motor1"]).mean().reset_index()

In [ ]:
# hack debut sm2


print(f"Wavelength used: {piston_imdata.wavelength.unique()}")
minPos = piston_imdata[['motor1','motor2','motor3']]
minPos = minPos - minPos.min()
piston_imdata['relPos'] = minPos['motor1']

#piston = piston_imdata[piston_imdata.EE5>threshold]

piston_imdata["fit_flag"] = 0
piston_imdata["EE5_diff_flag"] = True

piston = piston_imdata

#piston = piston_imdata[piston_imdata.EE5>threshold]

# filtering the data

In [ ]:

if doNeighborFilter :
    tot = []
    for  group_name, series in piston.groupby(['wavelength','fiber']):
        series = neighbor_outlier_filter(series, "EE5", thres_neighbor, absolute=True)
        tot.append(series)
        piston = pd.concat(tot)
        

# Through focus fit 

In [ ]:
piston_fitdata = fitFocusData(piston[piston.EE5_diff_flag] , index=piston_index, doPlot=False, criterias=criterias, head=head )

piston_focusMap = getFocusMap(piston_fitdata.dropna(), index=piston_index, criterias=criterias)

piston_focusMap["experimentId"] = experimentId

if doSaveCsv:
    piston_fitdata.to_csv(f"{csvPath}imquality_{cam}_Exp{experimentId}_doBck{str(doBck)}_piston_fitdata_thres{str(int(threshold*100)).zfill(3)}{forest_prefix}.csv")
    piston_focusMap.to_csv(f"{csvPath}imquality_{cam}_Exp{experimentId}_doBck{str(doBck)}_piston_focusMap_thres{str(int(threshold*100)).zfill(3)}{forest_prefix}.csv")



# 2nd Moment calculation from SEP

In [ ]:
if "sep_x2" in piston_imdata.columns:
    piston_imdata["sep_2ndM"]= piston_imdata.apply(lambda x: np.mean([x["sep_x2"],x["sep_y2"]]) , axis=1)

In [ ]:
piston[piston.fit_flag >=0 & piston.EE5_diff_flag].motor1.min()

In [ ]:
waves = piston_fitdata.wavelength.unique()
fibers = piston_fitdata.fiber.unique()
print(fibers, waves)
fiber = fibers[-1]
wave = waves[-1]
#    group = piston_imdata.groupby(['wavelength','fiber']).get_group((wave,fiber))
plot_one_group(piston, wave, fiber, experimentId, dataPath)

piston_plane = getBestPlane(piston_focusMap.query("criteria == '%s'"%criteria),order=1, \
                        doPlot=True, exp=experimentId, plot_path=dataPath)


txtfile = f"SM1_{cam.upper()}_BestFocusPlane_Exp{experimentId}_doBck{str(doBck)}_{criteria}_thres{str(int(threshold*100)).zfill(3)}.dat"

foc = findMotorPos(piston_plane, inv_mat = invMat)

txt = f"{datetime.now()} \n"
txt += f"{csvPath}\n"
txt += f"ExpId {experimentId}\n"
txt += f"{invMat}\n"
txt += f"{foc}\n"
txt += f"xcu_{cam} motors moveCcd a={foc[0]:.2f} b={foc[1]:.2f} c={foc[2]:.2f} microns abs\n"

print(txt)

# debug B2 
#dataPath = "/drp/analysis/sm2/debugB2"

if doSave :
    text_file = open(csvPath+txtfile, "w")
    text_file.write(txt)
    text_file.close()

plot_prefix = f"Focus_Piston_plots_doBck{str(doBck)}_{criteria}_thres{str(int(threshold*100)).zfill(3)}"
title_suffix = f"a={foc[0]:.2f} b={foc[1]:.2f} c={foc[2]:.2f} microns"
title_suffix += f"- DetBox {piston.detBoxTemp.mean():.1f}K - ccdTemp {piston.ccdTemp.mean():.1f}K"
plot_groups(piston.sort_values("motor1")[piston.EE5_diff_flag], experimentId, dataPath, plot_prefix=plot_prefix, title_suffix=title_suffix,\
            col="fiber", hue="wavelength", criteria=criteria, doSave=doSave, verbose=True)
plot_groups(piston.sort_values("motor1")[piston.EE5_diff_flag], experimentId, dataPath, plot_prefix=plot_prefix, title_suffix=title_suffix,\
            col="wavelength", hue="fiber", criteria=criteria, doSave=doSave)

plot_prefix = f"Focus_Piston_fit_plots_doBck{str(doBck)}_{criteria}_thres{int(threshold*100):<3}"
plot_groups(piston_fitdata.sort_values("motor1"), experimentId, dataPath, plot_prefix=plot_prefix, title_suffix=title_suffix, \
            col="fiber", hue="wavelength", criteria=criteria, doSave=doSave)
plot_groups(piston_fitdata.sort_values("motor1"), experimentId, dataPath, plot_prefix=plot_prefix, title_suffix=title_suffix,\
            col="wavelength", hue="fiber", criteria=criteria, doSave=doSave)

In [ ]:
piston_focusMap["ccdTemp"] = piston.ccdTemp.mean()
piston_focusMap["detBoxTemp"] = piston.detBoxTemp.mean()


In [ ]:
fib_list = [2,339,650]
piston_focusMap[piston_focusMap.fiber.isin(fib_list)].set_index(["fiber","wavelength"]).sort_index()[["relPos", "criteria","value", "detBoxTemp", "ccdTemp"]]

In [ ]:
piston_focusMap[piston_focusMap.fiber.isin(fib_list)].to_excel(f"{imgPath}{cam.upper()}_Exp{experimentId}.xlsx")

In [ ]:
plot_groups(piston[piston.fiber.isin(fib_list)].sort_values("motor1")[piston.EE5_diff_flag], experimentId, dataPath, plot_prefix=plot_prefix, title_suffix=title_suffix,\
            col="fiber", hue="wavelength", criteria=criteria, doSave=doSave, verbose=True)